In [3]:
%pip install fuzzywuzzy
%pip install Levenshtein
%pip install yahoo-finance
%pip install yfinance



Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
     ---------------------------------------- 0.0/3.0 MB ? eta -:--:--
     ---------- ----------------------------- 0.8/3.0 MB 8.3 MB/s eta 0:00:01
     ---------------------------------------- 3.0/3.0 MB 16.1 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished

In [4]:
import pandas as pd
import numpy as np
from fuzzywuzzy import fuzz, process
import yfinance as yf
import plotly.express as px
from functools import lru_cache

In [119]:
df1 = pd.read_csv('./stock_part1.csv')
df2 = pd.read_csv('./stock_part2.csv')

df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 11 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   ID           200000 non-null  int64 
 1   Date         200000 non-null  object
 2   Ticker       200000 non-null  object
 3   Open         200000 non-null  object
 4   High         200000 non-null  object
 5   Low          200000 non-null  object
 6   Close        200000 non-null  object
 7   Adj Close    200000 non-null  object
 8   Volume       200000 non-null  object
 9   Dividend     200000 non-null  object
 10  Split Ratio  161520 non-null  object
dtypes: int64(1), object(10)
memory usage: 16.8+ MB


In [100]:
df1

,ID,Date,Ticker,Open,High,Low,Close,Adj Close,Volume,Dividend,Split Ratio
0,1,2023-09-03,meta,258.44,909.63,1380.05,"595,86",61.54,"8,846,040,350",3.57,1:1
1,2,"Aug 28, 2024",NVDA,678.96,725.03,185.51,295.83,843.78,7346935699,2.85,3:2
2,3,"May 10, 2022",INTU,1348.94,215.00,831.86,874.05,"$336,19","8,027,589,419",2.28,2:1
3,4,2017-12-26,NVDA,$186.39,1111.85,508.65,511.49,$952.63,1939.5M,4.51,3:2
4,5,"Jan 20, 2022",WORK,1425.94,692.20,340.75,$378.58,1493.44,8984007687,1.93,NaN
...,...,...,...,...,...,...,...,...,...,...,...
199995,199996,"Jul 27, 2019",TSLA,"269,17",144.45,"418,85",588.19,$893.89,4184.54M,"4,71",3:2
199996,199997,"Sep 13, 2021",VMW,315.78,262.78,"89,13","$172,09",659.42,"5,782,458,138",1.04,3:2
199997,199998,2016-08-19,aapl,327.66,897.77,"1402,05",140.90,904.11,3104296586,$3.21,NaN
199998,199999,2020-06-24,DJCU,141.99,739.41,285.76,$529.92,1201.64,45454067,1.51,NaN


In [ ]:

df1['Date_original'] = df1['Date']

df1['Date'] = pd.to_datetime(df1['Date'], format="mixed", errors='coerce')


invalid_dates = df1[df1['Date'].isna()]

invalid_values = df1.loc[invalid_dates.index, 'Date_original']


nerr = len(invalid_dates) # quantas datas invalidas tem, ex: 12/63/2000 ou 12/81/2023
percent = nerr / len(df1["Date"]) * 100
print(f"Percentual de erros: {percent}%")
print(f"Número de erros: {nerr}")

#invalid_values.to_csv('datas_invalidas.csv')

Percentual de erros: 1.015%
Número de erros: 2030


In [131]:
from datetime import datetime

future_dates = df1[df1['Date'] > datetime.now()]
print(datetime.now())

future_dates["Date"].sort_values(ascending=False)

2025-05-21 19:00:18.950559


162764   2025-12-05
185818   2025-12-05
154923   2025-12-05
141381   2025-12-05
160494   2025-12-05
            ...    
142220   2025-06-01
24293    2025-06-01
184347   2025-06-01
43317    2025-06-01
143042   2025-06-01
Name: Date, Length: 498, dtype: datetime64[ns]

In [8]:
df1["Ticker"] = df1["Ticker"].str.upper().str.strip()

ticker_counts = df1["Ticker"].value_counts()
unique_ticker = ticker_counts[ticker_counts > 10].index
unique_values = df1[df1["Ticker"].isin(unique_ticker)]
print (unique_ticker)

Index(['ZM', 'SQ', 'PYPL', 'SHOP', 'META', 'BABA', 'SNAP', 'WORK', 'AMD',
       'CSCO', 'ORCL', 'AAPL', 'INTC', 'TSLA', 'NVDA', 'LYFT', 'IBM', 'SPOT',
       'VMW', 'ADBE', 'TWTR', 'PINS', 'MSFT', 'INTU', 'GOOG', 'DOCU', 'NFLX',
       'CRM', 'AMZN', 'UBER', 'INTF', 'EMD'],
      dtype='object', name='Ticker')


In [10]:
ticker_counts = df1['Ticker'].value_counts()
print(ticker_counts.describe())

count    2604.000000
mean       76.804916
std       679.282297
min         1.000000
25%         2.000000
50%         3.000000
75%         5.000000
max      6747.000000
Name: count, dtype: float64


In [ ]:
#Acho melhor baixar um df com os tickers válidos, para não precisar fazer requisição na api do yahoo
df1["Ticker"] = df1["Ticker"].str.upper().str.strip()


ticker_counts = df1["Ticker"].value_counts()


valid_tickers = list(set(
    ticker_counts[ticker_counts > 10].index.tolist()
))
valid_tickers = [t.upper() for t in valid_tickers]


@lru_cache(maxsize=None)
def is_valid_ticker(ticker):
    try:
        t = yf.Ticker(ticker)
        hist = t.history(period="1d")  
        return not hist.empty
    except:
        return False


def get_fuzzy_match(ticker, candidates, min_score=65):
    if not ticker or len(ticker) < 2:
        return None
    candidates_filtered = [t for t in candidates if abs(len(t) - len(ticker)) <= 1]
    if not candidates_filtered:
        return None
    match, score = process.extractOne(ticker, candidates_filtered)
    if score >= min_score:
        return match
    return None


corrections = {}
for ticker in ticker_counts[ticker_counts <= 10].index:
    if not is_valid_ticker(ticker):
        corrected = get_fuzzy_match(ticker, valid_tickers)
        if corrected:
            corrections[ticker] = corrected

print("Correções feitas:")
for k, v in corrections.items():
    print(f"{k} → {v}")


df1["Ticker"] = df1["Ticker"].replace(corrections)

$RBM: possibly delisted; no price data found  (period=1d)
$BMZN: possibly delisted; no price data found  (period=1d) (Yahoo error = "No data found, symbol may be delisted")
$INTB: possibly delisted; no price data found  (period=1d) (Yahoo error = "No data found, symbol may be delisted")
$AMZB: possibly delisted; no price data found  (period=1d)
$DOCG: possibly delisted; no price data found  (period=1d)
$IDM: possibly delisted; no price data found  (period=1d)
$IAPL: possibly delisted; no price data found  (period=1d) (Yahoo error = "No data found, symbol may be delisted")
$ADFE: possibly delisted; no price data found  (period=1d) (Yahoo error = "No data found, symbol may be delisted")
$AFZN: possibly delisted; no price data found  (period=1d) (Yahoo error = "No data found, symbol may be delisted")
$AMW: possibly delisted; no price data found  (period=1d)
$UBJR: possibly delisted; no price data found  (period=1d) (Yahoo error = "No data found, symbol may be delisted")
$AMZ: possibly del

In [110]:
numerics_columns = ["Open", "High", "Low", "Close", "Adj Close", "Volume", "Dividend"]
def get_non_numeric_characters(column):
  print(column.name)
  tot = set("")
  for index, value in column.items():
    tot = tot | set([c for c in value if not c.isdigit()])
  print(tot)

def count_comma(column):
    count = 0
    for index,value in column.items():
        if isinstance(value, str) and len(value) >= 3:
            if value[-3] in [',', '.']:
                count += 1
    print(count)



for i in numerics_columns:
  get_non_numeric_characters(df1[i])
  count_comma(df1[i])



Open
{'.', '$', ',', 'Z'}
200000
High
{'.', '$', ',', 'Z'}
200000
Low
{'.', '$', ',', 'Z'}
200000
Close
{'.', '$', ',', 'Z'}
200000
Adj Close
{'.', '$', ',', 'Z'}
200000
Volume
{'.', 'M', 'o', 'l', 'B', 'i', 'K', ' ', 'b', 'n', ','}
3503
Dividend
{'.', '$', ',', 'Z'}
200000


In [111]:
equivalents_numeric_columns = numerics_columns
equivalents_numeric_columns.remove("Volume")

def change_caracter (value):
  value = value.strip("$")
  value = value.strip()
  value = value.replace("Z", "2")
  value = value.replace(".", ",")
  return value

for i in equivalents_numeric_columns:
  df1[i] = df1[i].apply(change_caracter)

In [112]:
for i in equivalents_numeric_columns:
    df1[i] = df1[i].apply(
        lambda value: value[:-3] + "." + value[-2:] if len(value) >= 3 else value
    )
    df1[i] = df1[i].astype(float)   

In [121]:
df1

,ID,Date,Ticker,Open,High,Low,Close,Adj Close,Volume,Dividend,Split Ratio,Date_original
0,1,2023-09-03,meta,258.44,909.63,1380.05,"595,86",61.54,"8,846,040,350",3.57,1:1,2023-09-03
1,2,2024-08-28,NVDA,678.96,725.03,185.51,295.83,843.78,7346935699,2.85,3:2,"Aug 28, 2024"
2,3,2022-05-10,INTU,1348.94,215.00,831.86,874.05,"$336,19","8,027,589,419",2.28,2:1,"May 10, 2022"
3,4,2017-12-26,NVDA,$186.39,1111.85,508.65,511.49,$952.63,1939.5M,4.51,3:2,2017-12-26
4,5,2022-01-20,WORK,1425.94,692.20,340.75,$378.58,1493.44,8984007687,1.93,NaN,"Jan 20, 2022"
...,...,...,...,...,...,...,...,...,...,...,...,...
199995,199996,2019-07-27,TSLA,"269,17",144.45,"418,85",588.19,$893.89,4184.54M,"4,71",3:2,"Jul 27, 2019"
199996,199997,2021-09-13,VMW,315.78,262.78,"89,13","$172,09",659.42,"5,782,458,138",1.04,3:2,"Sep 13, 2021"
199997,199998,2016-08-19,aapl,327.66,897.77,"1402,05",140.90,904.11,3104296586,$3.21,NaN,2016-08-19
199998,199999,2020-06-24,DJCU,141.99,739.41,285.76,$529.92,1201.64,45454067,1.51,NaN,2020-06-24
